# Alpha Vantage Setup Guide

**Provider:** Alpha Vantage (Stock prices, fundamentals, forex, crypto)

**Best for:** Free stock data, fundamental data, technical indicators

**Time:** 3 minutes

**Cost:** FREE tier (25 requests/day), Premium from $49/month



## Step 1: Get Your Free API Key

1. Go to [alphavantage.co/support/#api-key](https://www.alphavantage.co/support/#api-key)
2. Fill in the form (name, email, use case)
3. Click **"Get Free API Key"**
4. **Copy the key** (shown immediately)

```
Format: XXXXXXXXXXXXXXXX (16 characters)
```

**No credit card required!**

**NOTE**: For premium access, go to https://www.alphavantage.co/premium/

## Step 2: Set Your API Key

Run this cell and paste your API key when prompted:

In [1]:
import os
from getpass import getpass

# Prompt for API key (hidden input)
api_key = getpass("Enter your Alpha Vantage API key: ")
os.environ["ALPHA_VANTAGE_API_KEY"] = api_key

print("✅ API key set!")
print(f"   Key length: {len(api_key)} characters")

Enter your Alpha Vantage API key: ··········
✅ API key set!
   Key length: 16 characters


## Step 3: Install Required Packages

In [2]:
!pip install -q requests pandas

## Step 4: Test Your Connection - Stock Quote

In [3]:
import requests

def get_stock_quote(ticker: str) -> dict:
    """Get current stock quote from Alpha Vantage."""
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "GLOBAL_QUOTE",
        "symbol": ticker,
        "apikey": os.environ["ALPHA_VANTAGE_API_KEY"]
    }
    response = requests.get(url, params=params)
    return response.json()

# Test with Apple
result = get_stock_quote("AAPL")

if "Global Quote" in result and result["Global Quote"]:
    quote = result["Global Quote"]
    print("✅ Connection successful!")
    print(f"   Ticker: {quote.get('01. symbol', 'N/A')}")
    print(f"   Price: ${quote.get('05. price', 'N/A')}")
    print(f"   Change: {quote.get('10. change percent', 'N/A')}")
elif "Note" in result:
    print("⚠️ Rate limit reached (25 calls/day on free tier)")
    print(f"   Message: {result['Note']}")
else:
    print("❌ Connection failed")
    print(f"   Response: {result}")

✅ Connection successful!
   Ticker: AAPL
   Price: $255.5300
   Change: -1.0379%


## Step 5: Test Historical Data

In [4]:
import pandas as pd

def get_daily_prices(ticker: str, outputsize: str = "compact") -> pd.DataFrame:
    """Get daily historical prices.

    Args:
        ticker: Stock symbol
        outputsize: 'compact' (100 days) or 'full' (20+ years)
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": outputsize,
        "apikey": os.environ["ALPHA_VANTAGE_API_KEY"]
    }
    response = requests.get(url, params=params)
    data = response.json()

    if "Time Series (Daily)" in data:
        df = pd.DataFrame(data["Time Series (Daily)"]).T
        df.columns = ["open", "high", "low", "close", "volume"]
        df = df.astype(float)
        df.index = pd.to_datetime(df.index)
        return df.sort_index()
    return pd.DataFrame()

# Get AAPL daily prices
df = get_daily_prices("AAPL")

if not df.empty:
    print("✅ Historical data retrieved!")
    print(f"   Period: {df.index[0].date()} to {df.index[-1].date()}")
    print(f"   Records: {len(df)}")
    print(f"\nLast 5 days:")
    print(df.tail())
else:
    print("❌ Failed to retrieve data (may be rate limited)")

✅ Historical data retrieved!
   Period: 2025-08-26 to 2026-01-16
   Records: 100

Last 5 days:
              open    high     low   close      volume
2026-01-12  259.16  261.30  256.80  260.25  45263767.0
2026-01-13  258.72  261.81  258.39  261.05  45730847.0
2026-01-14  259.49  261.82  256.71  259.96  40019421.0
2026-01-15  260.65  261.04  257.05  258.21  39388564.0
2026-01-16  257.90  258.90  254.93  255.53  72142773.0


## Step 6: Test Fundamental Data (Company Overview)

In [5]:
def get_company_overview(ticker: str) -> dict:
    """Get company fundamentals."""
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": os.environ["ALPHA_VANTAGE_API_KEY"]
    }
    response = requests.get(url, params=params)
    return response.json()

# Get Apple fundamentals
overview = get_company_overview("AAPL")

if overview and "Symbol" in overview:
    print("✅ Fundamental data retrieved!")
    print(f"   Company: {overview.get('Name', 'N/A')}")
    print(f"   Sector: {overview.get('Sector', 'N/A')}")
    print(f"   P/E Ratio: {overview.get('PERatio', 'N/A')}")
    print(f"   Market Cap: ${float(overview.get('MarketCapitalization', 0))/1e9:.1f}B")
    print(f"   Dividend Yield: {overview.get('DividendYield', 'N/A')}")
    print(f"   52-Week High: ${overview.get('52WeekHigh', 'N/A')}")
    print(f"   52-Week Low: ${overview.get('52WeekLow', 'N/A')}")
else:
    print("❌ Failed to retrieve fundamentals (may be rate limited)")

✅ Fundamental data retrieved!
   Company: Apple Inc
   Sector: TECHNOLOGY
   P/E Ratio: 34.25
   Market Cap: $3775.6B
   Dividend Yield: 0.004
   52-Week High: $288.62
   52-Week Low: $168.63


## Step 7: Create a Tool for smolagents

In [6]:
!pip install -q smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 2.4 MB/s eta 0:00:00


In [7]:
from smolagents import tool

@tool
def get_stock_price_av(ticker: str) -> dict:
    """Get current stock price and daily change from Alpha Vantage.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')

    Returns:
        Dictionary with price, change, and change_percent
    """
    import requests
    import os

    url = "https://www.alphavantage.co/query"
    params = {
        "function": "GLOBAL_QUOTE",
        "symbol": ticker.upper(),
        "apikey": os.environ["ALPHA_VANTAGE_API_KEY"]
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Global Quote" in data and data["Global Quote"]:
        quote = data["Global Quote"]
        return {
            "ticker": quote.get("01. symbol"),
            "price": float(quote.get("05. price", 0)),
            "change": float(quote.get("09. change", 0)),
            "change_percent": quote.get("10. change percent", "0%"),
            "volume": int(quote.get("06. volume", 0)),
            "source": "Alpha Vantage"
        }

    return {"error": "Failed to retrieve data", "ticker": ticker}

# Test the tool
result = get_stock_price_av("MSFT")
print("Tool test:")
print(result)

Tool test:
{'ticker': 'MSFT', 'price': 459.86, 'change': 3.2, 'change_percent': '0.7007%', 'volume': 34246650, 'source': 'Alpha Vantage'}


## ✅ Setup Complete!

You're ready to use Alpha Vantage in the course notebooks.

**Next steps:**
- Save your API key securely (password manager)
- Remember: 25 requests/day on free tier
- Use yfinance for bulk historical data (no limits)
